In [1]:
import pandas as pd
import os
import itertools
from pyspark.sql import SparkSession
import time
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import uuid
from spark_helper.core import create_spark_session



In [2]:
DATA_DIR = "/home/jovyan/data"
RESULTS_DIR = "/home/jovyan/results/classification"

In [3]:
# clean up the results directory
if os.path.exists(RESULTS_DIR):
    for item in os.listdir(RESULTS_DIR):
        item_path = os.path.join(RESULTS_DIR, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            import shutil
            shutil.rmtree(item_path)


In [4]:
# .config(("spark.driver.maxResultSize", "4g")) \
# Create a Spark session
# spark = SparkSession.builder \
#     .appName("SparkLightgbmClassification") \
#     .master(os.environ.get("SPARK_MASTER", "spark://spark-master:7077")) \
#     .config("spark.executor.memory", "4g") \
#     .config("spark.driver.maxResultSize", "1g") \
#     .config("spark.python.worker.faulthandler.enabled", "true") \
#     .config("spark.sql.execution.pyspark.udf.faulthandler.enabled", "true") \
#     .getOrCreate()
spark = create_spark_session("spark_cluster.yaml")

# get spark context
sc = spark.sparkContext

print(f"Spark version: {spark.version}")
print(f"Spark UI available at: {spark.sparkContext.uiWebUrl}")

Spark version: 4.0.0-preview2
Spark UI available at: http://2c39b2b1ca38:4040


In [5]:
# get unique fold ids from the spark DataFrame
fold_ids = [1, 2, 3, 4]

fold_ids

[1, 2, 3, 4]

In [6]:
def train_lightgbm_model(params=None):
    """
    Train a LightGBM classification model with the specified parameters.
    
    Parameters:
    -----------
    params : dict, optional
        Parameters for LGBMClassifier. If None, default parameters will be used.
        
    Returns:
    --------
    str
        File path where results are stored
    """
    
    print(f"Training LightGBM classification model...{params}")
    # Default parameters if none provided
    if params is None:
        raise ValueError("No parameters provided for LightGBM model.")
    
    fold_id = params.pop("fold_id")

    test_df = pd.read_parquet(
        os.path.join(DATA_DIR, f"ts_fold_{fold_id}_test.parquet"),
    )
    train_df = pd.read_parquet(
        os.path.join(DATA_DIR, f"ts_fold_{fold_id}_train.parquet"),
    )

    train_x = train_df.drop(columns=["target", "date"])
    train_y = train_df["target"]
    test_x = test_df.drop(columns=["target", "date"])
    test_y = test_df["target"]

    # Initialize the classification model
    model = LGBMClassifier(**params)
    
    # Train the model and measure time
    start_time = time.perf_counter()
    model.fit(
        train_x, train_y,
        eval_set=[(test_x, test_y)],
        eval_metric='logloss',
        callbacks=[
            lgb.early_stopping(stopping_rounds=2000),
            lgb.log_evaluation(period=100),
        ]
    )
    end_time = time.perf_counter()
    training_time = end_time - start_time
    
    # Make predictions
    y_pred_proba = model.predict_proba(test_x)[:, 1]  # Probability of positive class
    y_pred = model.predict(test_x)  # Class predictions
    
    # Calculate classification metrics
    accuracy = accuracy_score(test_y, y_pred)
    precision = precision_score(test_y, y_pred)
    recall = recall_score(test_y, y_pred)
    f1 = f1_score(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred_proba)

    # Store detailed classification report as string
    class_report = classification_report(test_y, y_pred)
    print(f"\nClassification Report for fold {fold_id}:")
    print(class_report)

    results = {
        "fold_id": fold_id,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc,
        'training_time': training_time,
    }
    results.update(params)

    # generate uuid string for the results file name
    fp_id = str(uuid.uuid4())
    fp_name = os.path.join(RESULTS_DIR, f"results_fold_{fold_id}_{fp_id}.parquet")

    results_df = pd.DataFrame([results])
    results_df.to_parquet(
        fp_name,
        index=False,
    )

    return fp_name

In [7]:
# Define the parameter grid for hyperparameter tuning

param_grid = {
    "n_jobs": [2],
    "fold_id": fold_ids,
    "lambda_l1": [0.3, 12, 40],
    "max_depth": [10, 15, 17],
    "colsample_bytree": [0.1, 0.3, 0.4],
    "alpha": [0.2],
    "num_leaves": [2048,],
    "learning_rate": [0.03],
    "lambda_l2": [0.01, 0.1],
    "max_bin": [256,],
    "bagging_fraction": [1],
    "deterministic": [False],
    "objective": ["huber"],
    "metric": ["huber"],
    "n_estimators": [20000],
    "random_state": [42],
    "importance_type": ["gain"],
}


# Generate all combinations of parameters
param_keys = list(param_grid.keys())
param_values = list(param_grid.values())
param_combinations = list(itertools.product(*param_values))

# Create a list of dictionaries, each representing a specific combination
param_dicts = []
for combo in param_combinations:
    param_dict = dict(zip(param_keys, combo))
    param_dicts.append(param_dict)

# Display the number of combinations and the first few combinations
print(f"Total number of parameter combinations: {len(param_dicts)}")
print("\nFirst 3 parameter combinations:")
for i in range(min(3, len(param_dicts))):
    print(f"Combination {i+1}:")
    print(param_dicts[i])


Total number of parameter combinations: 216

First 3 parameter combinations:
Combination 1:
{'n_jobs': 2, 'fold_id': 1, 'lambda_l1': 0.3, 'max_depth': 10, 'colsample_bytree': 0.1, 'alpha': 0.2, 'num_leaves': 2048, 'learning_rate': 0.03, 'lambda_l2': 0.01, 'max_bin': 256, 'bagging_fraction': 1, 'deterministic': False, 'objective': 'huber', 'metric': 'huber', 'n_estimators': 20000, 'random_state': 42, 'importance_type': 'gain'}
Combination 2:
{'n_jobs': 2, 'fold_id': 1, 'lambda_l1': 0.3, 'max_depth': 10, 'colsample_bytree': 0.1, 'alpha': 0.2, 'num_leaves': 2048, 'learning_rate': 0.03, 'lambda_l2': 0.1, 'max_bin': 256, 'bagging_fraction': 1, 'deterministic': False, 'objective': 'huber', 'metric': 'huber', 'n_estimators': 20000, 'random_state': 42, 'importance_type': 'gain'}
Combination 3:
{'n_jobs': 2, 'fold_id': 1, 'lambda_l1': 0.3, 'max_depth': 10, 'colsample_bytree': 0.3, 'alpha': 0.2, 'num_leaves': 2048, 'learning_rate': 0.03, 'lambda_l2': 0.01, 'max_bin': 256, 'bagging_fraction': 1, 

In [8]:
%%time
process_these = param_dicts[:10]  # Limit to the first 10 combinations for testing

rdd = sc.parallelize(process_these, numSlices=len(process_these))

rdd_result = rdd.map(lambda x: train_lightgbm_model(params=x)).collect()

print(f">>>>length of rdd_result: {len(rdd_result)}")


>>>>length of rdd_result: 10
CPU times: user 12.4 ms, sys: 3.57 ms, total: 16 ms
Wall time: 49.2 s


In [9]:
df_list = [pd.read_parquet(fp) for fp in rdd_result]
results_df = pd.concat(df_list, ignore_index=True)

results_df.head

<bound method NDFrame.head of    fold_id  accuracy  precision    recall  f1_score   auc_roc  training_time  \
0        1   0.85750   0.929913  0.701590  0.799775  0.940603      23.300903   
1        1   0.86115   0.934811  0.707013  0.805109  0.942692      21.284032   
2        1   0.92600   0.952517  0.860471  0.904157  0.967199      29.583189   
3        1   0.92855   0.952478  0.867127  0.907801  0.967815      26.705687   
4        1   0.93405   0.952571  0.881302  0.915552  0.967652      36.456858   
5        1   0.93280   0.953747  0.876864  0.913691  0.967790      36.798010   
6        1   0.84525   0.921822  0.675829  0.779888  0.933211      26.306204   
7        1   0.84790   0.917917  0.686429  0.785472  0.932914      21.543370   
8        1   0.91940   0.954171  0.841735  0.894434  0.966156      21.920865   
9        1   0.91560   0.954449  0.831628  0.888816  0.966006      20.701826   

   n_jobs  lambda_l1  max_depth  ...  learning_rate  lambda_l2  max_bin  \
0       2     

In [10]:
results_df[["training_time", "accuracy", "precision", "recall", "f1_score", "auc_roc"]].describe()

,training_time,accuracy,precision,recall,f1_score,auc_roc
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,26.460094,0.896820,0.942440,0.792999,0.859469,0.955204
std,6.067711,0.038402,0.014743,0.087927,0.058527,0.015646
min,20.701826,0.845250,0.917917,0.675829,0.779888,0.932914
25%,21.637744,0.858413,0.931138,0.702946,0.801109,0.941125
50%,24.803554,0.917500,0.952498,0.836682,0.891625,0.966081
75%,28.863814,0.927913,0.953453,0.865463,0.906890,0.967539
max,36.798010,0.934050,0.954449,0.881302,0.915552,0.967815


In [11]:
spark.stop()